# Deep Learning Day2 for Chanwoo Lee

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

In [6]:
samples = 1000
test_samples = 100
train_dataset = './ffnn_dataset/train_dataset.csv'
test_dataset = './ffnn_dataset/test_dataset.csv'

def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])

    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
write_dataset(1000, 100, train_dataset, test_dataset)

train data is written
test data is written


In [7]:
tf.reset_default_graph()
trainset = tf.contrib.data.TextLineDataset(train_dataset).batch(10)
testset = tf.contrib.data.TextLineDataset(test_dataset).batch(10)

train_itr = trainset.make_one_shot_iterator()
test_itr = testset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

train_batch = tf.decode_csv(train_batch, [[0]]*11)
test_batch = tf.decode_csv(test_batch, [[0]]*11)

y_ = tf.reshape(train_batch[0], [-1, 1])
test_label = tf.reshape(test_batch[0], [-1, 1])

x = tf.stack(train_batch[1:], axis=1)
test_data = tf.stack(test_batch[1:], axis=1)

test_label = tf.cast(test_label, tf.float32)
test_input = tf.cast(test_data, tf.float32)
y_ = tf.cast(y_, tf.float32)
x = tf.cast(x, tf.float32)

print(y_)
print(x)
print(test_label)
print(test_input)

Tensor("Cast_2:0", shape=(?, 1), dtype=float32)
Tensor("Cast_3:0", shape=(?, 10), dtype=float32)
Tensor("Cast:0", shape=(?, 1), dtype=float32)
Tensor("Cast_1:0", shape=(?, 10), dtype=float32)


In [8]:
def bin_model(x, activation, reuse=False):
    layer1 = tf.layers.dense(x, 10, activation=activation, reuse=reuse, name='layer1')
    layer2 = tf.layers.dense(layer1, 10, activation=activation, reuse=reuse, name='layer2')
    layer3 = tf.layers.dense(layer2, 10, activation=activation, reuse=reuse, name='layer3')
    layer4 = tf.layers.dense(layer3, 10, activation=activation, reuse=reuse, name='layer4')
    return tf.layers.dense(layer4, 1, activation=activation, reuse=reuse, name='layer_out')


train_out = bin_model(x, tf.nn.relu)
test_out = bin_model(test_input, tf.nn.relu, True)

for var in tf.trainable_variables():
    print(var)

<tf.Variable 'layer1/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'layer1/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'layer2/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'layer2/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'layer3/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'layer3/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'layer4/kernel:0' shape=(10, 10) dtype=float32_ref>
<tf.Variable 'layer4/bias:0' shape=(10,) dtype=float32_ref>
<tf.Variable 'layer_out/kernel:0' shape=(10, 1) dtype=float32_ref>
<tf.Variable 'layer_out/bias:0' shape=(1,) dtype=float32_ref>


In [9]:
loss = tf.losses.sigmoid_cross_entropy(y_, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

pred = tf.nn.sigmoid(test_out)
accuracy = tf.metrics.accuracy(y_, tf.round(pred))

saver = tf.train.Saver()

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.


In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for i in range(10):
        while True:
            try:
                _, _loss = sess.run([train_op, loss])
                _acc = sess.run(accuracy)
            except tf.errors.OutOfRangeError:
                break
                
        print('epoch: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))
        saver.save(sess, './logs/model')

epoch: 0, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 1, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 2, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 3, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 4, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 5, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 6, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 7, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 8, loss: 0.6931471824645996, acc: 0.3888888955116272
epoch: 9, loss: 0.6931471824645996, acc: 0.3888888955116272
